<a href="https://colab.research.google.com/github/atjoelpark/eutilities/blob/main/demo/PubMedExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Obtaining Keys from the US National Library of Medicine
# Link: https://www.ncbi.nlm.nih.gov/home/develop/api/

In [2]:
# The two APIs of interest for PubMed Articles
# 1. Entrez Programming Utilities
# 2. PubMed Central (PMC) APIs

# Website provides documentation in regards to scope and uses.

In [3]:
# Use of APIs are free, but please do not abuse the services.
# 1. Do not run API requests concurrently.
# 2. Include parameters to help identify what services you require.
# 2a. 'tool' should be the name of your application, as a string value with no internal spaces.
# 2b. 'email' should be the email address of the maintainer of the tool